In [378]:
import os
import random
from random import randint
import numpy as np
import pandas as pd
import scipy as sc

from gensim.models import Word2Vec
import networkx as nx

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets

# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
# seed the global RNG 
random.seed(SEED)
# seed the global NumPy RNG
np.random.seed(SEED)

In [2]:
# define models path name
path = "../Embedding_models/"
models = ['FPBitsEmbedding',
        'FPCountsEmbedding',
        'MolDescEmbedding',
        'QuantumPropsEmbedding'] 
pnames = [path + p for p in models]

In [3]:
def find_similar(data, threshold=0.9, similar=True):
  """
  this function computes the most or least similar molecules given a similarity threshold
  :param data: dataframe of molecules with cid as index
  :param threshold: similarity value
  :param similar: boolean indicating whether to return the most or least similar items
  :param seed: seed for reproducibility 
  :return: list of similar molecules
  """
  
  # define sign function 
  if (similar):
    def sign_func(x,y): return x > y
  else: 
    def sign_func(x,y): return x < y
  
  top_similar = pd.DataFrame()
  n=len(data)
  # iterate over molecules
  for molecule_id in data.index:
      df_similar = pd.DataFrame()
      # load embedding model
      for PATH in pnames:
        model = Word2Vec.load(PATH)
        # check if key is present
        try:
          similar = [item for item in model.wv.most_similar(str(molecule_id), topn=n) if sign_func(item[1], threshold)]
          df = pd.DataFrame(similar)     
        except KeyError:  
          continue
        
        # check if result is empty
        if not(df.empty):
          # set columns
          df.columns=['similar_cid','measure']
          # generate output dataframe
          df_similar = pd.concat([df_similar, df])
          # check if similar id is in input data 
          df_similar = df_similar[df_similar['similar_cid'].isin(data.index.astype(str))]  
          # round similarity values
          df_similar['measure'] = df_similar['measure'].round(3)
      
      # check if result is empty
      if not(df_similar.empty):
          out_df = pd.DataFrame()
          # sort similar molecules by frequency
          similar_freq = df_similar.groupby(['similar_cid'])['similar_cid'].count().reset_index(name='Count').sort_values(['Count'], ascending=False)
          for i in range(len(similar_freq)):
              # get info of most frequent molecules 
              tmp_df = df_similar.loc[df_similar['similar_cid'] == similar_freq['similar_cid'].iloc[i]]
              tmp_df['orig_cid'] = molecule_id
              out_df = pd.concat([out_df, tmp_df])
          # generate output dataframe
          top_similar = pd.concat([out_df, top_similar])

  return top_similar
    

In [4]:
# load targets table 
targets = pd.read_csv('../Flask/psql_db/Tables/TargetsTable.csv', encoding='latin1', index_col=0)
targets.head()

,Chemical_Name,Molecule_CID,Target_GI,Target_GeneID,Target_Symbol,UniProtKB,PDB,Resolution
APDB_idx,,,,,,,,
T.1,"1,2,4-Trichlorobenzene",13,325495497.0,6256,RXRA,Q6P3U7,NaN,NaN
T.2,"1,2,4-Trichlorobenzene",13,325495463.0,5914,RARA,P10276,1DSZ,1.7
T.3,"1,2,4-Trichlorobenzene",13,119601739.0,7253,TSHR,A0A0A0MTJ0,NaN,NaN
T.4,"1,2,4-Trichlorobenzene",13,54288833.0,2100,ESR2,F1D8N3,NaN,NaN
T.5,"1,2,4-Trichlorobenzene",13,348019627.0,2099,ESR1,G4XH65,NaN,NaN


In [6]:
# check if there are null symbols
targets.Target_Symbol.isnull().values.any()

False

In [7]:
# get list and count of targets for each molecule
mol_targets = targets.groupby('Molecule_CID')['Target_Symbol'].apply(list).reset_index(name='Target_Symbol')
mol_targets['#Targets'] = targets.groupby('Molecule_CID')['Target_Symbol'].apply(len).values
# sort values 
mol_targets.sort_values(by=['#Targets'], inplace=True)
mol_targets.reset_index(drop=True, inplace=True)
mol_targets

,Molecule_CID,Target_Symbol,#Targets
0,7245,[ESR1],1
1,6416,[CYP1A2],1
2,6419,[CYP1A2],1
3,7962,[CYP1A2],1
4,6495,[CYP2C19],1
...,...,...,...
532,6001,"[HIF1A, APLNR, NLRP3, MDM4, MDM2, OPRD1, OPRM1...",38
533,289,"[HSP90AA1, HSP90AB1, ALOX15, HPGD, HIF1A, ALDH...",40
534,17520,"[MAPT, S1PR4, KCNJ1, CACNA1B, KCNQ1, KAT2A, BA...",43
535,33528,"[CYP3A4, HCRTR1, MDM4, MDM2, CCR6, VDR, HSP90A...",48


In [8]:
# load molecules table 
molecules = pd.read_csv('../Flask/psql_db/Tables/MoleculesTable.csv', encoding='latin1')
molecules.head()

,Chemical_Name,CID,CAS,InChIKey,Canonical_SMILES,Molecular_Formula,Atoms,Molecule_Type
0,(1-methylpropyl)benzene,8680,135-98-8,ZJMWRROPUADPEA-UHFFFAOYSA-N,CCC(C)C1=CC=CC=C1,C10H14,[C],organic
1,(2-methylbutyl)cyclohexane,143127,54105-77-0,DDQXBDLAGHZBMP-UHFFFAOYSA-N,CCC(C)CC1CCCCC1,C11H22,[C],organic
2,(2-methylpropyl)benzene,10870,538-93-2,KXUHSQYYJYAXGZ-UHFFFAOYSA-N,CC(C)CC1=CC=CC=C1,C10H14,[C],organic
3,"1,1,1-trichloroethane",6278,71-55-6,UOCLXMDMGBRAIB-UHFFFAOYSA-N,CC(Cl)(Cl)Cl,C2H3Cl3,"[C, Cl]",organic
4,"1,1,2,2-tetrachloroethane",6591,79-34-5,QPFMBZIOSGYJDE-UHFFFAOYSA-N,C(C(Cl)Cl)(Cl)Cl,C2H2Cl4,"[C, Cl]",organic


In [35]:
# plot percentage of organic and inorganic molecules
counts_type = pd.DataFrame({'Type': ['organic', 'inorganic'], 
                        'Total': [(molecules['Molecule_Type'] == "organic").sum(), (molecules['Molecule_Type'] == "inorganic").sum()]})
# pie chart 
fig = px.pie(counts_type, values = 'Total', names = 'Type', title='Molecule types', hole=.4, color = 'Type', width=400, height=400)

# update layout
fig.update_layout(title_x=0.45, title_font_size=14)

# save and show
fig.write_html("../Flask/pollutants_db_app/static/img/molecule_type.html", config={'displaylogo': False})
fig.show(config={'displaylogo': False})

In [10]:
# merge dataframes 
mol_targets = mol_targets.merge(molecules, how='inner', left_on='Molecule_CID', right_on='CID').set_index('CID') 
mol_targets.head()

,Molecule_CID,Target_Symbol,#Targets,Chemical_Name,CAS,InChIKey,Canonical_SMILES,Molecular_Formula,Atoms,Molecule_Type
CID,,,,,,,,,,
7245,7245,[ESR1],1,2-Chlorophenol,95-57-8,ISPYQTSUDJAMAB-UHFFFAOYSA-N,C1=CC=C(C(=C1)O)Cl,C6H5ClO,"[C, O, Cl]",organic
6416,6416,[CYP1A2],1,"2-Butanone, 3,3-dimethyl-",75-97-8,PJGSXYOJTGTZAV-UHFFFAOYSA-N,CC(=O)C(C)(C)C,C6H12O,"[C, O]",organic
6419,6419,[CYP1A2],1,Pentachloroethane,76-01-7,BNIXVQGCZULYKV-UHFFFAOYSA-N,C(C(Cl)(Cl)Cl)(Cl)Cl,C2HCl5,"[C, Cl]",organic
7962,7962,[CYP1A2],1,Methylcyclohexane,108-87-2,UAEPNZWRGJTJPN-UHFFFAOYSA-N,CC1CCCCC1,C7H14,[C],organic
6495,6495,[CYP2C19],1,Dimethoxypropane,77-76-9,HEWZVZIVELJPQZ-UHFFFAOYSA-N,CC(C)(OC)OC,C5H12O2,"[C, O]",organic


In [42]:
# plot the distribution of # targets associated to each molecule
fig = px.histogram(mol_targets, x="#Targets", color="Molecule_Type", 
                    width=750, height=400)

# update layout
fig.update_xaxes(type='category', tickmode='linear', tickangle=90)
fig.update_layout(
    title="Counts of molecules for each group of associated targets",
    title_font_size = 14,
    yaxis_title="#Molecules",
    legend_title="molecule type",
    legend_font_size = 12)

# configuration
config = {'modeBarButtonsToRemove': ['logo', 'zoomin', 'zoomout', 'zoom', 
          'lasso', 'pan', 'autoscale', 'select'], 'displaylogo': False}

# save and show 
fig.write_html("../Flask/pollutants_db_app/static/img/distribution_molecules_targets.html", config=config)
fig.show(config=config)

#### Distribution of similar molecules given a threshold 

In [184]:
threshold = [0.95, 0.85, 0.75]
out_similar = []
for thr in threshold:
    # find similar molecues
    top_similar = find_similar(mol_targets, threshold=thr)

    # construct set of edges
    edges = list(set(top_similar[["orig_cid", "similar_cid"]].astype(int).itertuples(index=False, name=None)))

    # construct graph
    G = nx.Graph()
    G.add_edges_from(edges)
    print("\nGraph thr.:", thr, "\n#Edges:", len(G.edges()), "\n#Nodes:", len(G.nodes()))

    # compute #neighbors
    neigh = []
    for n in G.nodes:
        neigh.append((n, len([n for _, n in G.edges(n)])))

    # generate output dataframe
    df_similar = pd.DataFrame({'cid': [t[0] for t in neigh], '#similar': [t[1] for t in neigh]})
    print("Min:", df_similar['#similar'].min(), "\nMax:", df_similar['#similar'].max())

    out_similar.append(df_similar)



Graph thr.: 0.95 
#Edges: 6189 
#Nodes: 526
Min: 1 
Max: 78

Graph thr.: 0.85 
#Edges: 18011 
#Nodes: 535
Min: 6 
Max: 173

Graph thr.: 0.75 
#Edges: 31634 
#Nodes: 535
Min: 11 
Max: 236


In [185]:
# return least similar molecules with threshold 0.75
unsimilar = find_similar(mol_targets, threshold=0.75, similar=False, seed=SEED)
# construct set of edges
edges = list(set(unsimilar[["orig_cid", "similar_cid"]].astype(int).itertuples(index=False, name=None)))
edges[0:5]

[(13387, 8461), (8146, 87244), (7005, 10522), (379, 6561), (991, 24121)]

In [186]:
# construct graph for sim < 0.75
G = nx.Graph()
G.add_edges_from(edges)
print("#Edges:", len(G.edges()), "\n#Nodes:", len(G.nodes()))

# compute #neighbors
neigh = []
for n in G.nodes:
   neigh.append((n, len([n for _, n in G.edges(n)])))

# generate output dataframe
df_unsimilar = pd.DataFrame({'cid': [t[0] for t in neigh], '#similar': [t[1] for t in neigh]})
print("Min:", df_unsimilar['#similar'].min(), "\nMax:", df_unsimilar['#similar'].max())

# add to output list
out_similar.append(df_unsimilar)

#Edges: 96322 
#Nodes: 535
Min: 186 
Max: 477


In [193]:
# make subplots
fig = make_subplots(rows=2, cols=2, x_title='#Similar Molecules', y_title='#Molecules') 
# > 0.95
trace1 = go.Histogram(name=">0.95", x=out_similar[0]['#similar'], nbinsx=int(out_similar[0]['#similar'].max()), marker=go.histogram.Marker(color='#D04CF4'))
# > 0.85
trace2 = go.Histogram(name=">0.85", x=out_similar[1]['#similar'], nbinsx=int(out_similar[1]['#similar'].max()), marker=go.histogram.Marker(color='#724CF4'))
# > 0.75
trace3 = go.Histogram(name=">0.75", x=out_similar[2]['#similar'], nbinsx=int(out_similar[2]['#similar'].max()), marker=go.histogram.Marker(color='#45DA3E'))
# < 0.75
trace4 = go.Histogram(name="<0.75", x=out_similar[3]['#similar'], nbinsx=int(out_similar[3]['#similar'].max()), marker=go.histogram.Marker(color='#F4D34C'))

# update layout
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.update_layout(title=dict(text="Counts of molecules for each group of similar molecules at different similarity thr.", font_size=14),
                legend_title="similarity thr.", width=750)
# update x and y labels
fig.layout.annotations[0].update(font_size=14)
fig.layout.annotations[1].update(font_size=14)

# configuration
config = {'modeBarButtonsToRemove': ['logo', 'zoomin', 'zoomout', 'zoom', 
    'lasso', 'pan', 'autoscale', 'select'], 'displaylogo': False}

# save and show 
fig.write_html("../Flask/pollutants_db_app/static/img/distribution_neighbors_similar_molecules.html", config=config)
fig.show(config=config)

##### Similarity assessment

In [393]:
def similarity_assessment(data, thr1, thr2, n_perm=5, seed=SEED):
    """
    this function 
    :param data: dataframe of molecules with associated targets and cid as index
    :param g1: threshold for most similar molecules
    :param g2: threshold for least similar molecules
    :param seed: seed for reproducibility 
    :return: 
    """

    # seed the global RNG 
    random.seed(seed)
    np.random.seed(seed)

    # most similar molecules
    top_similar = find_similar(data, threshold=thr1, seed=SEED)
    print("Min. Similarity:", top_similar.measure.min())
    # construct set of edges
    edges = list(set(top_similar[["orig_cid", "similar_cid"]].astype(int).itertuples(index=False, name=None)))
    # construct graph 
    G = nx.Graph()
    G.add_edges_from(edges)
    print("\n#Nodes:", len(G.nodes()), "\n#Edges:", len(G.edges()))

    # set targets as node attribute
    targets = []
    nx.set_node_attributes(G, targets, "targets")
    for n in G.nodes:
        G.nodes[n]["targets"] = data.Target_Symbol[data.index == n].values[0]

    # least similar molecules
    top_unsimilar = find_similar(data, threshold=thr2, similar=False, seed=SEED)
    print("\n\nMin. Similarity:", top_unsimilar.measure.min())
    # construct set of edges
    un_edges = list(set(top_unsimilar[["orig_cid", "similar_cid"]].astype(int).itertuples(index=False, name=None)))
    # construct graph 
    un_G = nx.Graph()
    un_G.add_edges_from(un_edges)
    print("\n#Nodes:", len(un_G.nodes()), "\n#Edges:", len(un_G.edges()))

    # set targets as node attribute
    targets = []
    nx.set_node_attributes(un_G, targets, "targets")
    for n in un_G.nodes:
        un_G.nodes[n]["targets"] = data.Target_Symbol[data.index == n].values[0]

    stats = []
    figures = []
    for _ in range(n_perm):
        
        if len(G.edges) < len(un_G.edges):
            G1 = G.copy()
            # get same number of edges/pairs of molecules 
            selected_edges = random.sample(un_G.edges, len(G.edges))
            # construct subgraph 
            G2 = un_G.edge_subgraph(selected_edges)

        else:
            G2 = un_G.copy()
            # get same number of edges/pairs of molecules 
            selected_edges = random.sample(G.edges, len(un_G.edges))
            # construct subgraph 
            G1 = G.edge_subgraph(selected_edges)
            
        # common targets between source molecule and similar molecules
        visited_pair = []
        similar_targets, similar_no_targets = 0, 0 
        for n in G1.nodes:
            # get similar molecules
            neighbors_nodes = [n for _, n in G1.edges(n)]

            for neigh in neighbors_nodes:
                # add visited pair
                visited_pair.append((n,neigh))

                if not (neigh,n) in visited_pair:
                    # check common targets
                    if len(np.intersect1d(G1.nodes[n]["targets"], G1.nodes[neigh]["targets"])) > 0:
                        similar_targets+=1
                    else: similar_no_targets+=1

        # common targets between source molecule and non similar molecules
        visited_pair = []
        no_similar_targets, no_similar_no_targets = 0, 0
        for n in G2.nodes:
            # get similar molecules
            neighbors_nodes = [n for _, n in G2.edges(n)]

            for neigh in neighbors_nodes:
                # add visited pair
                visited_pair.append((n,neigh))

                if not (neigh,n) in visited_pair:
                    # check common targets
                    if len(np.intersect1d(G2.nodes[n]["targets"], G2.nodes[neigh]["targets"])) > 0:
                        no_similar_targets+=1
                    else: no_similar_no_targets+=1

        # create a dataframe of booleans
        dct = {
            'similar': [True] * (similar_targets + similar_no_targets) + [False] * (no_similar_targets + no_similar_no_targets),
            'targets': [True] * (similar_targets) + [False] * (similar_no_targets) + [True] * (no_similar_targets) + [False] * (no_similar_no_targets),
        }
        df = pd.DataFrame(dct)
        
        # construct the pivot table
        pivot_table = pd.pivot_table(df, index='similar', columns='targets', aggfunc='size')
        # re-format the pivot table into a contingency table
        contingency_table = pivot_table.iloc[::-1, ::-1]
        
        # compute chi2 
        chi2stats = sc.stats.chi2_contingency(contingency_table)
        # compute odds ratio
        oddsratio, pval = sc.stats.fisher_exact(contingency_table)
        # output stats
        stats.append((contingency_table, chi2stats, (oddsratio, pval)))

        # plot contingency table and statistics
        fig = px.imshow(contingency_table, text_auto=True, 
                    labels={'color': '#pairs'}, width=440, height=520,
                    title="Chi-square test and odds ratio <br><sup><br>Chi2: "+
                    str(round(chi2stats[0], 2))+", p-Value: "+str(chi2stats[1])+
                    "<br>OddsR: "+str(round(oddsratio, 2))+", p-Value: "+str(pval)+
                    "</sup><br>", color_continuous_scale=px.colors.sequential.Viridis_r,)
        fig.update_layout(title_x=0.2)
        fig.update(layout_coloraxis_showscale=False)
        # save figure
        figures.append(fig)
        
    return stats, figures

In [394]:
# run statistics
stats, figures = similarity_assessment(mol_targets, 0.96, 0.46, n_perm=6, seed=SEED)
stats

Min. Similarity: 0.96

#Nodes: 513 
#Edges: 5080


Min. Similarity: 0.192

#Nodes: 535 
#Edges: 48325


[(targets  True   False
  similar              
  True      2428   2652
  False     2130   2950,
  (35.09855049260756,
   3.134340184591768e-09,
   1,
   array([[2279., 2801.],
          [2279., 2801.]])),
  (1.2679950998095157, 3.101515232351809e-09)),
 (targets  True   False
  similar              
  True      2428   2652
  False     2182   2898,
  (23.835922690586465,
   1.0490627272581433e-06,
   1,
   array([[2305., 2775.],
          [2305., 2775.]])),
  (1.2159586248657257, 1.0440355978503634e-06)),
 (targets  True   False
  similar              
  True      2428   2652
  False     2145   2935,
  (31.623654158573345,
   1.8713621578493866e-08,
   1,
   array([[2286.5, 2793.5],
          [2286.5, 2793.5]])),
  (1.2527256554405877, 1.855484027562807e-08)),
 (targets  True   False
  similar              
  True      2428   2652
  False     2140   2940,
  (32.761535117718836,
   1.041861998563021e-08,
   1,
   array([[2284., 2796.],
          [2284., 2796.]])),
  (1.2577916860489702,

In [395]:
# make subplots
fig1 = []
fig2 = []
for i in range(int(len(figures)/2)): fig1.append(go.FigureWidget(figures[i].data, layout=figures[i].layout))
for i in range(int(len(figures)/2), int(len(figures))): fig2.append(go.FigureWidget(figures[i].data, layout=figures[i].layout))
widgets.VBox([widgets.HBox(fig1), widgets.HBox(fig2)])

    'data': [{'coloraxis': 'coloraxis',
              'hovertempl…

In [396]:
# run statistics
stats, figures = similarity_assessment(mol_targets, 0.98, 0.44, n_perm=6, seed=SEED)
stats

Min. Similarity: 0.98

#Nodes: 453 
#Edges: 2805


Min. Similarity: 0.192

#Nodes: 535 
#Edges: 40696


[(targets  True   False
  similar              
  True      1323   1482
  False     1177   1628,
  (15.170450160771704,
   9.822862964293943e-05,
   1,
   array([[1250., 1555.],
          [1250., 1555.]])),
  (1.2347799765409209, 9.788072038362995e-05)),
 (targets  True   False
  similar              
  True      1323   1482
  False     1211   1594,
  (8.86779974402739,
   0.002902447573614022,
   1,
   array([[1267., 1538.],
          [1267., 1538.]])),
  (1.1750485595937377, 0.002899027970587217)),
 (targets  True   False
  similar              
  True      1323   1482
  False     1203   1602,
  (10.197860538339018,
   0.0014060368382486446,
   1,
   array([[1263., 1542.],
          [1263., 1542.]])),
  (1.1887992569184327, 0.0014038275083555147)),
 (targets  True   False
  similar              
  True      1323   1482
  False     1204   1601,
  (10.026471166221544,
   0.00154306379222785,
   1,
   array([[1263.5, 1541.5],
          [1263.5, 1541.5]])),
  (1.187070426513511, 0.001540

In [397]:
# make subplots
fig1 = []
fig2 = []
for i in range(int(len(figures)/2)): fig1.append(go.FigureWidget(figures[i].data, layout=figures[i].layout))
for i in range(int(len(figures)/2), int(len(figures))): fig2.append(go.FigureWidget(figures[i].data, layout=figures[i].layout))
widgets.VBox([widgets.HBox(fig1), widgets.HBox(fig2)])

    'data': [{'coloraxis': 'coloraxis',
              'hovertempl…

The association between rows (similar) and columns (targets) is statistically significant. Once we have determined the probability that the two variables are related (using the Chi-square test), we explore their relationship in more detail with the odds ratio (is an event more or less likely to occur in one condition or another?).<br>
The odds ratio is greater than 1, then being similar is considered to be associated with having common targets since being similar raises the odds of having common targets. Therefore, the group of similar molecules has slightly higher odds of having common targets than non similar molecules.